In [1]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import *
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
# reading wsdm csv
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")

# reading members csv
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")

# merging train dataset with members dataset
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')

# getting the head (top 5 rows) of df
wsdm_members.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0


In [3]:
### 4.1 Loading files to merge-

In [4]:
# wsdm data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")
# members data
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")
# transactions data
transactions_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\transactions_v2.csv")
# user logs data
logs_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\user_logs_v2.csv")

In [5]:
### 4.2 Merging files together-
### 4.2.1 For train data-

In [6]:
# merging members data with train data
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')
# merging transactions data with train members data
wsdm_mem_trans = pd.merge(wsdm_members, transactions_data, on='msno', how='left')
# merging user logs data with train members transaction data
wsdm_dataset = pd.merge(wsdm_mem_trans, logs_data, on='msno', how='left')
# getting head of train dataset
pd.set_option('display.max_columns', 100)
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170305.0,7.0,0.0,3.0,5.0,71.0,68.0,17599.893
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170301.0,138.0,19.0,7.0,1.0,21.0,158.0,8830.433
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170319.0,0.0,0.0,0.0,0.0,34.0,17.0,7883.313
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170316.0,15.0,0.0,0.0,1.0,38.0,17.0,9029.227
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170310.0,0.0,0.0,0.0,0.0,8.0,8.0,1870.110


In [7]:
wsdm_dataset.shape

(16887877, 23)

In [8]:
# since registration_init_time contains a lot of nan values so impute median date
wsdm_dataset['registration_init_time'] = wsdm_dataset['registration_init_time'].fillna(20150204.0)
# getting registration_init_time array
reg_date = wsdm_dataset['registration_init_time'].values
reg_date

array([20131223., 20131223., 20131223., ..., 20110905., 20110905.,
       20150204.])

In [9]:
# storing these train registrationn dates to npz file
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\wsdm_registration_date", reg_date)

<h3> Vorverarbeitung
   

In [10]:
def preprocess(data):
    '''This preprocess function is used to perform basic preprocessing on top of train and test dataset
    These preprocessing consist outliers removal, nan imputation and replacing the values'''
    
    # imputing 0 in place of nan values in the city column
    data['city'] = data['city'].fillna(0)

    # removing outliers
    data['bd'] = data['bd'].apply(lambda x: x if (x < 72.0) and (x > 0.0) else np.nan)
    # imputing 28 as age instead of nan
    data['bd'] = data['bd'].fillna(28.0)

    # replacing male with 1 in gender
    data['gender'] = data['gender'].replace(to_replace='male', value=1)
    # replacing male with 2 in gender
    data['gender'] = data['gender'].replace(to_replace='female', value=2)
    # replacing nan with 0 in gender
    data['gender'] = data['gender'].fillna(0)

    # replace 0 instead of nan in registered_via
    data['registered_via'] = data['registered_via'].fillna(0)

    # filling median date in place of nan in the df
    data['registration_init_time'] = data['registration_init_time'].fillna(20150204.0)
    # converting float date to datetime
    data['registration_init_time'] = pd.to_datetime(data['registration_init_time'], format='%Y%m%d')

    # imputing 0 in place of nan value in payment_method_id
    data['payment_method_id'] = data['payment_method_id'].fillna(0)

    # removing outliers
    data['payment_plan_days'] = data['payment_plan_days'].apply(lambda x: x if (x <= 30.0) else np.nan)
    # imputing 30 in place of nan in payment_plan_days
    data['payment_plan_days'] = data['payment_plan_days'].fillna(30.0)

    # removing outliers
    data['plan_list_price'] = data['plan_list_price'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in plan_list_price
    data['plan_list_price'] = data['plan_list_price'].fillna(149.0)

    # removing outliers
    data['actual_amount_paid'] = data['actual_amount_paid'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in actual_amount_paid
    data['actual_amount_paid'] = data['actual_amount_paid'].fillna(149.0)

    # imputing 2 in place of nan values in is_auto_renew
    data['is_auto_renew'] = data['is_auto_renew'].fillna(2)

    # filling median date in place of nan in the df
    data['transaction_date'] = data['transaction_date'].fillna(20170316.0)
    # converting float date to datetime
    data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='%Y%m%d')

    # filling median date in place of nan in the df
    data['membership_expire_date'] = data['membership_expire_date'].fillna(20170419.0)
    # converting float date to datetime
    data['membership_expire_date'] = pd.to_datetime(data['membership_expire_date'], format='%Y%m%d')

    # imputing 2 in place of nan values in is_cancel
    data['is_cancel'] = data['is_cancel'].fillna(2)

    # filling median date in place of nan in the df
    data['date'] = data['date'].fillna(20170316.0)
    # converting float date to datetime
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')

    # removing outliers
    data['num_25'] = data['num_25'].apply(lambda x: x if (x <= 15.0) else np.nan)
    # now I can impute 2 instead of nan in num_25
    data['num_25'] = data['num_25'].fillna(2.0)

    # removing outliers
    data['num_50'] = data['num_50'].apply(lambda x: x if (x <= 4.0) else np.nan)
    # now I can impute 0 instead of nan in num_50
    data['num_50'] = data['num_50'].fillna(0)

    # removing outliers
    data['num_75'] = data['num_75'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_75
    data['num_75'] = data['num_75'].fillna(0)

    # removing outliers
    data['num_985'] = data['num_985'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_985
    data['num_985'] = data['num_985'].fillna(0)

    # removing outliers
    data['num_100'] = data['num_100'].apply(lambda x: x if (x <= 74.0) else np.nan)
    # now I can impute 14 instead of nan in num_100
    data['num_100'] = data['num_100'].fillna(14.0)

    # removing outliers
    data['num_unq'] = data['num_unq'].apply(lambda x: x if (x <= 68.0) else np.nan)
    # now I can impute 16 instead of nan in num_unq
    data['num_unq'] = data['num_unq'].fillna(16.0)

    # removing outliers
    data['total_secs'] = data['total_secs'].apply(lambda x: x if (x <= 19167.549700000025) else np.nan)
    # now I can impute 3880.765 instead of nan in total_secs
    data['total_secs'] = data['total_secs'].fillna(3880.765)

In [11]:
# performing preprocessing on top of entire train data
preprocess(wsdm_dataset)

In [12]:
# again just to cross check if there is any null or not
wsdm_dataset.isnull().values.any()

False

In [13]:
# checking for any duplicate row
wsdm_dataset.duplicated().any()

False

In [14]:
# saving wsdm file to disk
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\train.csv", index=False)

<h2>Feature Engineering

In [15]:
# loading the files
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\train.csv")


In [16]:
# getting head of wsdm file
pd.set_option('display.max_columns', 100)
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-05,7.0,0.0,3.0,0.0,71.0,68.0,17599.893
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-01,2.0,0.0,0.0,1.0,21.0,16.0,8830.433
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-19,0.0,0.0,0.0,0.0,34.0,17.0,7883.313
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-16,15.0,0.0,0.0,1.0,38.0,17.0,9029.227
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-10,0.0,0.0,0.0,0.0,8.0,8.0,1870.110


In [17]:
# all features in wsdm data
wsdm_dataset.columns

Index(['msno', 'is_churn', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')

In [18]:
# reference: https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e
# reference: https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html
# reference: https://numpy.org/doc/stable/reference/generated/numpy.finfo.html

def datatype_changer(dataset):
    # iterating through all the columns in the dataframe
    for col in dataset.columns:
        # getting column's datatype
        col_type = dataset[col].dtype
        
        # checking if datatype of column is 'object' or not
        # if column type is not object
        if (col_type == int) or (col_type == float):
            # getting minimum value of a column
            min_val = dataset[col].min()
            # getting maximum value of a column
            max_val = dataset[col].max()
            # checking whether the datatype contain first 3 characters as int or not, if int
            if str(col_type)[:3] == 'int':
                # cheking the minimal and maximal value for int8, int16, int32 and int64 in numpy
                if min_val > np.iinfo(np.int8).min and max_val < np.iinfo(np.int8).max:
                    dataset[col] = dataset[col].astype(np.int8)
                elif min_val > np.iinfo(np.int16).min and max_val < np.iinfo(np.int16).max:
                    dataset[col] = dataset[col].astype(np.int16)
                elif min_val > np.iinfo(np.int32).min and max_val < np.iinfo(np.int32).max:
                    dataset[col] = dataset[col].astype(np.int32)
                else:
                    dataset[col] = dataset[col].astype(np.int64)
            else:
                # if it is non int, which is ultimately float
                # cheking the minimal and maximal value for float16, float32 and float64 in numpy
                if min_val > np.finfo(np.float16).min and max_val < np.finfo(np.float16).max:
                    dataset[col] = dataset[col].astype(np.float16)
                elif min_val > np.finfo(np.float32).min and max_val < np.finfo(np.float32).max:
                    dataset[col] = dataset[col].astype(np.float32)
                else:
                    dataset[col] = dataset[col].astype(np.float64)
        else:
            # keeping rest of them to category datatype instead of object
            dataset[col] = dataset[col].astype('category')
            
    # returning head of the dataframe
    return dataset

In [19]:
wsdm_dataset = datatype_changer(wsdm_dataset)

In [20]:
# saving train file
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\wsdm_FA.csv", index=False)

<h1>3. Prepare Data for Modelling

In [2]:
# loading train dataset
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\wsdm_FA.csv")
# setting max_columns to 100, so it displays all the columns of train dataset
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of train dataset
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-05,7.0,0.0,3.0,0.0,71.0,68.0,17600.0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-01,2.0,0.0,0.0,1.0,21.0,16.0,8830.0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-19,0.0,0.0,0.0,0.0,34.0,17.0,7884.0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-16,15.0,0.0,0.0,1.0,38.0,17.0,9030.0
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,2013-12-23,0.0,30.0,149.0,149.0,2.0,2017-03-16,2017-04-19,2.0,2017-03-10,0.0,0.0,0.0,0.0,8.0,8.0,1870.0


In [3]:
# Konvertiere Datum für Input Rohdatenmatrix
wsdm_dataset['date'] = wsdm_dataset['date'].str.replace('-', '').astype(float)
wsdm_dataset['membership_expire_date'] = wsdm_dataset['membership_expire_date'].str.replace('-', '').astype(float)
wsdm_dataset['transaction_date'] = wsdm_dataset['transaction_date'].str.replace('-', '').astype(float)

In [4]:
# getting shape of train dataset
wsdm_dataset.shape

(16887877, 23)

In [5]:
# adding registration init time feature to wsdm dataset
# load the registration array
reg_date = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\wsdm_registration_date.npz")
# display the array
reg_date['arr_0']

array([20131223., 20131223., 20131223., ..., 20110905., 20110905.,
       20150204.])

In [6]:
# converting numpy array to list
registration_init_time = reg_date['arr_0'].tolist()
# getting length of the list
len(registration_init_time)

16887877

In [7]:
# creating new feature, and adding it to wsdm dataset
wsdm_dataset['registration_init_time'] = registration_init_time

In [8]:
# setting max_columns to 100, so it displays all the columns of wsdm dataset
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of wsdm dataset
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170305.0,7.0,0.0,3.0,0.0,71.0,68.0,17600.0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170301.0,2.0,0.0,0.0,1.0,21.0,16.0,8830.0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170319.0,0.0,0.0,0.0,0.0,34.0,17.0,7884.0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170316.0,15.0,0.0,0.0,1.0,38.0,17.0,9030.0
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170310.0,0.0,0.0,0.0,0.0,8.0,8.0,1870.0


In [9]:
# saving new dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\prepared_train.csv", index=False)

In [10]:
# reading csv file for removing duplicates
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\prepared_train.csv")
# getting all columns of train dataset
wsdm_dataset.columns

Index(['msno', 'is_churn', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')

In [11]:
# # removing all duplicate rows for a particular id
# wsdm_dataset = wsdm_dataset.drop_duplicates('msno', keep='first', inplace=False)
# # setting max_columns to 100, so it displays all the columns of train dataset
# pd.set_option('display.max_columns', 100)
# # displaying head (top 10) rows of train dataset
# wsdm_dataset.head(10)

In [12]:
# getting shape of wsdm dataset
wsdm_dataset.shape

(16887877, 23)

In [13]:
# getting columns and converting them to list
cols = wsdm_dataset.columns.tolist()
# saving them for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\columns.npz", cols)

In [14]:
# saving actual wsdm dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_train.csv", index=False)

<h2> 3.2 Train Test Split

In [15]:
# loading actual wsdm data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_train.csv")
# setting max_columns to 100, so it displays all the columns of train data
pd.set_option('display.max_columns', 100)
# displaying head (top 5) rows of wsdm data
wsdm_data.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170305.0,7.0,0.0,3.0,0.0,71.0,68.0,17600.0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170301.0,2.0,0.0,0.0,1.0,21.0,16.0,8830.0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170319.0,0.0,0.0,0.0,0.0,34.0,17.0,7884.0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170316.0,15.0,0.0,0.0,1.0,38.0,17.0,9030.0
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,1.0,3.0,20131223.0,0.0,30.0,149.0,149.0,2.0,20170316.0,20170419.0,2.0,20170310.0,0.0,0.0,0.0,0.0,8.0,8.0,1870.0


In [16]:
# checking for null values
wsdm_data.isnull().values.any()

False

In [17]:
# replacing all nan and infinity to 0
wsdm_data = wsdm_data.replace(np.nan, 0, inplace=False)
wsdm_data = wsdm_data.replace(np.inf, 0, inplace=False)

In [18]:
## Sample 50% of Data due to memory restrictions
wsdm_data = wsdm_data.sample(frac=.5)

In [19]:
# https://stackoverflow.com/a/29651514
def normalize(df):
    result1 = df.copy()
    for feature_name in df.columns:
        if (str(feature_name) != str('msno') and str(feature_name)!=str('is_churn')):
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result1[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result1


wsdm_dataset = normalize(wsdm_data)

In [20]:
# getting labels for y
labels = wsdm_dataset['is_churn'].values
labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
# getting labels for X
data = wsdm_dataset.drop(['msno', 'is_churn'], axis=1, inplace=False)
data

,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
9516294,0.590909,0.342857,1.0,0.692308,0.158286,0.829268,1.0,0.827778,0.827778,0.5,1.000000,0.003350,0.0,0.266667,0.133333,0.75,0.000000,0.000000,0.918919,0.223881,0.863850
1584328,0.045455,0.385714,0.0,0.538462,0.695942,1.000000,1.0,0.550000,0.550000,0.5,0.999654,0.003251,0.0,0.466667,0.200000,0.50,0.333333,0.000000,0.189189,0.223881,0.202400
5281487,0.590909,0.385714,0.5,0.692308,0.543794,0.804878,1.0,0.827778,0.827778,0.5,1.000000,0.003350,0.0,0.000000,0.133333,0.00,0.000000,0.000000,0.081081,0.089552,0.083464
15777914,0.363636,0.371429,0.5,0.538462,0.697504,1.000000,1.0,0.716667,0.716667,0.5,0.999011,0.003037,0.0,0.200000,0.000000,0.00,0.000000,0.000000,0.040541,0.029851,0.045175
10878112,0.045455,0.385714,0.0,0.538462,0.852113,1.000000,1.0,0.827778,0.827778,0.5,0.998665,0.002921,0.0,0.900000,0.266667,0.25,0.000000,0.000000,0.445946,0.477612,0.381325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16086030,0.045455,0.385714,0.0,0.692308,0.772155,0.926829,1.0,0.827778,0.827778,0.0,0.999753,0.003268,0.0,0.200000,0.200000,0.75,0.666667,1.000000,0.297297,0.164179,0.324674
1035822,0.045455,0.385714,0.0,1.000000,0.998377,0.731707,1.0,0.827778,0.827778,0.5,0.999011,0.001386,0.5,0.366667,0.133333,0.75,0.666667,0.333333,0.229730,0.432836,0.245175
13381453,0.045455,0.385714,0.0,0.538462,0.852213,1.000000,1.0,0.550000,0.550000,0.5,0.999357,0.003152,0.0,0.400000,0.466667,0.25,0.333333,0.000000,0.094595,0.208955,0.100104
4679300,1.000000,0.371429,0.5,0.230769,0.616153,0.951220,1.0,0.827778,0.827778,0.5,1.000000,0.004803,0.0,0.900000,0.666667,0.00,0.666667,0.666667,0.229730,0.432836,0.251956


In [22]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 21) (5910756,)
(2533182, 21) (2533182,)


In [23]:
# converting X_train to numpy array
wsdm_arr = X_train.values
wsdm_arr

array([[0.04545455, 0.38571429, 0.        , ..., 0.54054054, 0.91044776,
        0.59467917],
       [0.04545455, 0.38571429, 0.        , ..., 0.35135135, 0.32835821,
        0.36932704],
       [0.63636364, 0.38571429, 1.        , ..., 0.01351351, 0.        ,
        0.01054768],
       ...,
       [0.04545455, 0.38571429, 0.        , ..., 0.2027027 , 0.26865672,
        0.18622844],
       [0.18181818, 0.37142857, 0.5       , ..., 0.5       , 0.6119403 ,
        0.54094938],
       [0.18181818, 0.37142857, 0.5       , ..., 0.01351351, 0.01492537,
        0.02097021]])

In [39]:
# checking shape of train array
wsdm_arr.shape

(5910756, 21)

In [25]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_train.npz", wsdm_arr)

In [26]:
# displaying y_train
wsdm_labels_arr = y_train
wsdm_labels_arr

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [27]:
# checking shape of train labels
wsdm_labels_arr.shape

(5910756,)

In [28]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_wsdm_labels.npz", wsdm_labels_arr)

In [29]:
# converting X_test to numpy array
X_arr = X_test.values
X_arr

array([[0.68181818, 0.38571429, 0.        , ..., 0.08108108, 0.07462687,
        0.08085546],
       [0.04545455, 0.38571429, 0.        , ..., 0.02702703, 0.02985075,
        0.02176312],
       [0.04545455, 0.38571429, 0.        , ..., 0.52702703, 0.59701493,
        0.53625454],
       ...,
       [0.04545455, 0.38571429, 0.        , ..., 0.12162162, 0.11940299,
        0.12592588],
       [0.04545455, 0.38571429, 0.        , ..., 0.47297297, 0.6119403 ,
        0.48951484],
       [0.04545455, 0.38571429, 0.        , ..., 0.17567568, 0.1641791 ,
        0.17235259]])

In [30]:
# checking shape of test array
X_arr.shape

(2533182, 21)

In [31]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv.npz", X_arr)

In [32]:
# displaying y_test
X_labels_arr = y_test
X_labels_arr

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [33]:
# checking shape of test labels array
X_labels_arr.shape

(2533182,)

In [34]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv_labels.npz", X_labels_arr)

<h1> 5. Modelling

In [41]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [42]:
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_train.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_wsdm_labels.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv_labels.npz")

In [43]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 21) (5910756,)
(2533182, 21) (2533182,)


In [44]:
# loading columns
cols = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\columns.npz")
# converting numpy array to python list
cols = cols['arr_0'].tolist()
# removing first two cols 'msno' and 'is_churn'
cols = cols[2:]

<h3> Logistic Regression

In [68]:
%%time

# fitting logistic regression model with best parameter values
lr_model = LogisticRegression(penalty='l2', tol=0.0001, C=10, solver='newton-cg', class_weight='balanced', n_jobs=-1)
lr_model.fit(X_train, y_train)

Wall time: 2min 14s


LogisticRegression(C=10, class_weight='balanced', n_jobs=-1, solver='newton-cg')

In [69]:
# Reference: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the trained logistic regression ML model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_lr.sav"
joblib.dump(lr_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_3\\finalized_model_lr.sav']

In [70]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_lr.sav"
loaded_model = joblib.load(filename)

<h3> Model Evaluation

In [71]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

lr_predict=lr_model.predict(X_test)
lr_predict


print('Accuracy:', accuracy_score(y_test, lr_predict))
print('F1 score weighted:', f1_score(y_test, lr_predict,average='weighted'))
print('F1 score macro:', f1_score(y_test, lr_predict, average='macro'))
print('Recall:', recall_score(y_test, lr_predict,average='weighted'))
print('Precision:', precision_score(y_test, lr_predict,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, lr_predict))
print('\n confussion matrix:\n',confusion_matrix(y_test, lr_predict))

Accuracy: 0.8273207373177293
F1 score weighted: 0.8509097408248447
F1 score macro: 0.6691593882381189
Recall: 0.8273207373177293
Precision: 0.8918860491383352

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.85      0.90   2272948
           1       0.33      0.66      0.44    260234

    accuracy                           0.83   2533182
   macro avg       0.64      0.75      0.67   2533182
weighted avg       0.89      0.83      0.85   2533182


 confussion matrix:
 [[1923621  349327]
 [  88101  172133]]


In [72]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.49378311939416736


<h2> Decision Tree

In [73]:
%%time

# training a decision tree classifier model with best parameters
dtc_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=5, min_samples_split=3, min_samples_leaf=1)
dtc_model.fit(X_train, y_train)

Wall time: 34.8 s


DecisionTreeClassifier(max_depth=5, min_samples_split=3)

In [74]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_dtc.sav"
joblib.dump(dtc_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_3\\finalized_model_dtc.sav']

In [75]:
# loading the best decision tree classifier model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_dtc.sav"
loaded_model = joblib.load(filename)

<h3>Model Evaluation

In [76]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

dtc_modelpred=dtc_model.predict(X_test)
dtc_modelpred


print('Accuracy:', accuracy_score(y_test, dtc_modelpred))
print('F1 score weighted:', f1_score(y_test, dtc_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, dtc_modelpred, average='macro'))
print('Recall:', recall_score(y_test, dtc_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, dtc_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, dtc_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, dtc_modelpred))

Accuracy: 0.9551725853097014
F1 score weighted: 0.9533674046400298
F1 score macro: 0.8687300489193444
Recall: 0.9551725853097014
Precision: 0.9529912079877343

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98   2272948
           1       0.84      0.70      0.76    260234

    accuracy                           0.96   2533182
   macro avg       0.90      0.84      0.87   2533182
weighted avg       0.95      0.96      0.95   2533182


 confussion matrix:
 [[2237634   35314]
 [  78242  181992]]


In [77]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.14579857183696102


## Random Forest

In [78]:
%%time

# training the random forest classifier for best hyper parameters
rf_model = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=2, max_features='sqrt', bootstrap=False, n_jobs=-1)
rf_model.fit(X_train, y_train)

Wall time: 26min 31s


RandomForestClassifier(bootstrap=False, max_depth=15, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=3,
                       n_estimators=500, n_jobs=-1)

In [79]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_rf.sav"
joblib.dump(rf_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_3\\finalized_model_rf.sav']

In [80]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_3\finalized_model_rf.sav"
loaded_model = joblib.load(filename)

<h3>Model Evaluation

In [81]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

rf_modelpred=rf_model.predict(X_test)
rf_modelpred


print('Accuracy:', accuracy_score(y_test, rf_modelpred))
print('F1 score weighted:', f1_score(y_test, rf_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, rf_modelpred, average='macro'))
print('Recall:', recall_score(y_test, rf_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, rf_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, rf_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, rf_modelpred))

Accuracy: 0.9650487015934899
F1 score weighted: 0.9636635850282442
F1 score macro: 0.8977695814709494
Recall: 0.9650487015934899
Precision: 0.9637491432419677

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   2272948
           1       0.89      0.75      0.81    260234

    accuracy                           0.97   2533182
   macro avg       0.93      0.87      0.90   2533182
weighted avg       0.96      0.97      0.96   2533182


 confussion matrix:
 [[2249833   23115]
 [  65423  194811]]


In [82]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.10445284741332707
